In [1]:
from __future__ import division
from nltk.corpus import conll2002
print "Number of words in the training set {}".format(len(conll2002.iob_words()))
print conll2002.iob_sents()[0]
train = conll2002.iob_words()
train_tags = [x[2] for x in train]

Number of words in the training set 678377
[(u'Sao', u'NC', u'B-LOC'), (u'Paulo', u'VMI', u'I-LOC'), (u'(', u'Fpa', u'O'), (u'Brasil', u'NC', u'B-LOC'), (u')', u'Fpt', u'O'), (u',', u'Fc', u'O'), (u'23', u'Z', u'O'), (u'may', u'NC', u'O'), (u'(', u'Fpa', u'O'), (u'EFECOM', u'NP', u'B-ORG'), (u')', u'Fpt', u'O'), (u'.', u'Fp', u'O')]


In [2]:
#calculate the conditional probability
from collections import Counter
import pickle
#because we're using the linear interpotation
#all trigram, bigram and unigram probabilities are needed here

trigrams, bigrams = [], []
# * => representing the start of the document
trigrams.append(('*', '*', train[0][2]))
trigrams.append(('*', train_tags[0], train_tags[1]))
i = 0
while i <= len(train_tags) - 3:
    trigrams.append((train_tags[i], train_tags[i+1], train_tags[i+2]))
    i += 1
trigram_counter = Counter(trigrams)

#same thing for bigram and unigram
bigrams.append(('*', train_tags[0]))
i = 0 
while i <= len(train_tags) - 3:
    bigrams.append((train_tags[i], train_tags[i+1]))
    i += 1
bigram_counter = Counter(bigrams)

unigram_counter = Counter(train_tags)

In [3]:
#construct the conditional probability
#q(s|u,v) = W1 * q'(s|u,v) + W2 * q'(s|u) + W3 * q'(s)
#where q' is the maximum likelihood of q
#and W1 + W2 + W3 = 1
def conditional_prob(trigram, w1=0.6, w2=0.2, w3=0.2):
    trigram_total = sum(trigram_counter.values())
    bigram_total = sum(bigram_counter.values())
    unigram_total = sum(unigram_counter.values())
    
    trigram_prob = trigram_counter[trigram]/trigram_total
    bigram_prob = bigram_counter[(trigram[1], trigram[2])]/bigram_total
    unigram_prob = unigram_counter[trigram[2]]/unigram_total
    return w1 * trigram_prob + w2 * bigram_prob + w3 * unigram_prob


In [4]:
import os
os.listdir('./')

['.ipynb_checkpoints',
 'bigram_counter.pkl',
 'CalcProb.py',
 'CalcProb.pyc',
 'CreateCorpus.py',
 'CreateCorpus.pyc',
 'HMM.py',
 'Tagging_HMM.ipynb',
 'trigram_counter.pkl',
 'unigram_counter.pkl']

In [5]:
#calculating the emission probability
#probability of word given tag
#p(word|tag) = C(word,tag)/C(tag)  C => Count
word_tag_count = Counter([(x[0],x[2]) for x in train])
tag_count = Counter(train_tags)

def emission_prob(word, tag):
    return word_tag_count[(word, tag)]/tag_count[tag]

In [50]:
tuple(test)

('w', 'g')